# Check Functions

In [34]:
from PyNetwork import ActivationFunctions, ErrorFunctions, MetricFunctions
import PyNetworkCPU
import pyopencl as cl
import pyopencl.array as cl_array
import numpy as np

platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)

x = np.random.rand(100, 20).astype(np.float32)
x_gpu = cl_array.to_device(queue, x.astype(np.float32))

gpu_activation = ActivationFunctions(context, queue)
gpu_error = ErrorFunctions()
gpu_metric = MetricFunctions(context, queue)

## Testing Activation Functions

In [35]:
linear_cpu = PyNetworkCPU.get_activation_function('linear')
linear_gpu = gpu_activation.get_activation_function('linear')

# Test the result of passing through
np.testing.assert_almost_equal(linear_cpu(x), linear_gpu(x_gpu).get())
np.testing.assert_almost_equal(linear_cpu(x, grad=True), linear_gpu(x_gpu, grad=True).get())

In [36]:
relu_cpu = PyNetworkCPU.get_activation_function('relu')
relu_gpu = gpu_activation.get_activation_function('relu')

# Test the result of ReLU
np.testing.assert_almost_equal(relu_cpu(x), relu_gpu(x_gpu).get())
np.testing.assert_almost_equal(relu_cpu(x, grad=True), relu_gpu(x_gpu, grad=True).get())

In [37]:
softmax_cpu = PyNetworkCPU.get_activation_function('softmax')
softmax_gpu = gpu_activation.get_activation_function('softmax')

# Test the result of softmax
np.testing.assert_almost_equal(softmax_cpu(x), softmax_gpu(x_gpu).get())
np.testing.assert_almost_equal(softmax_cpu(x, grad=True), softmax_gpu(x_gpu, grad=True).get())

## Testing Error Functions

In [38]:
mask = np.random.randint(2, size=(100, 20)).astype(np.int32)
target = mask * x
target_gpu = cl_array.to_device(queue, target.astype(np.float32))

In [39]:
mse_cpu = PyNetworkCPU.get_error_function('mse')
mse_gpu = gpu_error.get_error_function('mse')

# Test the result of passing through
np.testing.assert_almost_equal(mse_cpu(target, x), mse_gpu(target_gpu, x_gpu).get())
np.testing.assert_almost_equal(mse_cpu(x, target, grad=True), mse_gpu(x_gpu, target_gpu, grad=True).get(), decimal=6)

In [40]:
cross_entropy_cpu = PyNetworkCPU.get_error_function('cross_entropy')
cross_entropy_gpu = gpu_error.get_error_function('cross_entropy')

# Test the result of cross entropy
np.testing.assert_almost_equal(cross_entropy_cpu(x, target), cross_entropy_gpu(x_gpu, target_gpu).get(), decimal=3)
np.testing.assert_almost_equal(cross_entropy_cpu(x, target, grad=True), cross_entropy_gpu(x_gpu, target_gpu, grad=True).get(), decimal=3)

## Testing Metric Functions

In [41]:
accuracy_cpu = PyNetworkCPU.get_metric_function('accuracy')
accuracy_gpu = gpu_metric.get_metric_function('accuracy')

# Test the result of accuracy
np.testing.assert_almost_equal(accuracy_cpu(target, x), accuracy_gpu(target_gpu, x_gpu).get())